In [1]:
import shutil
import subprocess
import pandas as pd
import os
import datetime

# DeepLabCut
After training a DeepLabCut model, we used it to predict the spine points of all larvae in various videos. For each video, the output is a .h5 file containing the predicted coordinates for the video. The following code converts the .h5 file to a .spine file, which enables a direct comparison to the other investigated trackers.

In [80]:
def convert_DLC_h5_to_spine(h5_path, output_path):
    date_time = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    video_name = os.path.basename(h5_path).split('.')[0].split('DLC')[0]
    df = pd.read_hdf(h5_path)

    print(f'Processing {video_name}')

    # Open output file
    output_file = open(f'{output_path}/{video_name}.spine', 'w')

    individuals = df.columns.levels[1]
    # Iterate over individuals
    for individual in individuals:
        indiv_df = df['DLC_dlcrnetms5_Three_Selected_VideosJul19shuffle1_100000'][individual]
        # Iterate over rows in dataframe
        for index, row in indiv_df.iterrows():
            # Get larva id with leading zeros, the naming convention for the individual larvae may vary, hence the if-else
            if individual.startswith('ind'):
                larva_id = "{0:0>5}".format(int(individual[3:]))
            elif individual.startswith('larvae'):
                larva_id = "{0:0>5}".format(int(individual[6:]))
            else:
                raise ValueError('Unknown individual name')
            # Calculate time based on frame with 30 fps; with 3 digits after decimal point
            time = "{:.3f}".format((index + 1) / 30)

            coordinates = []
            for body_part in ['head-tip', 'head-lower', 'middle-head', 'middle', 'middle-tail', 'tail-lower', 'tail']:
                coordinates.extend([row[body_part]['x'], row[body_part]['y']])

            # If all coordinates are not NaN, write to output file
            if not 'nan' in [str(coord) for coord in coordinates]:
                out = [date_time, larva_id, time]
                out.extend(coordinates)
                output_file.write(' '.join(map(str, out)) + os.linesep)

    output_file.close()

In [81]:
input_dir = '/Path/to/DLC/output_files'
output_dir = '/Path/to/DLC/spine_files'
for file in os.listdir(input_dir):
    if file.endswith('filtered.h5'):
        video_name = file.split('DLC')[0]
        os.mkdir(os.path.join(output_dir, video_name))
        convert_DLC_h5_to_spine(os.path.join(input_dir, file), os.path.join(output_dir, video_name))

Processing A1DF31_A1_2022-07-12-150920-0000_ffv1
Processing A1DF35_A1_2022-07-07-164027-0000_ffv1
Processing A1DF39_B1_2022-07-12-185350-0000_ffv1
Processing A1DF38_A1_2022-07-12-194950-0000_ffv1
Processing A1DF33_B1_2022-07-07-153521-0000_ffv1
Processing A1DF36_A1_2022-07-07-142945-0000_ffv1
Processing A1DF33_A1_2022-07-07-153358-0000_ffv1
Processing A1DF35_B1_2022-07-07-164359-0000_ffv1
Processing A1DF39_A1_2022-07-12-184950-0000_ffv1
Processing A1DF36_B1_2022-07-07-143248-0000_ffv1
Processing A1DF34_A1_2022-07-07-133023-0000_ffv1
Processing A1DF32_B1_2022-07-12-161457-0000_ffv1
Processing A1DF32_A1_2022-07-12-161124-0000_ffv1
Processing A1DF37_A1_2022-07-06-093303-0000_ffv1
Processing A1DF33_A2_2022-07-07-153918-0000_ffv1
Processing A1DF37_B1_2022-07-06-093415-0000_ffv1
Processing A1DF34_B1_2022-07-07-133354-0000_ffv1
Processing A1DF38_B1_2022-07-12-195403-0000_ffv1


# MWT Tracker
We use the [MWT command line version](https://gitlab.com/larvataggerpipelines/mwt-cli) to process the videos. The tool must be installed first by cloning the linked repository. The installation instructions can be found in the repository. We expect a mwt-cli directory as working directory, which is simply the cloned repository.

In [4]:
# Change the following paths to your needs

# Directory containing all videos to be processed
video_dir = '/Path/to/Larvae_Videos'
# Path to the mwt-cli installation; Installation instructions: https://gitlab.com/larvataggerpipelines/mwt-cli
working_dir = '/Path/to/mwt-cli'
# Directory to save the output spine files, the output for each video is saved in a subdirectory named after the video
output_dir_mwt = '/Path/to/Larvae_Tracking_Benchmarking/Results/MWT'

In [6]:
# Optimized hyperparameters
pixel_thr1 = 231
pixel_thr2 = 249
size_thr1 = 15
size_thr2 = 15

date_time = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')

# Iterate over all videos in video directory and process them
for file in os.listdir(video_dir):
    video_path = os.path.join(video_dir, file)
    video_id = os.path.basename(video_path).split('.')[0]

    print(f'Processing {video_id}')
    
    # Create a subdirectory for the output files of the current video
    os.mkdir(f'{output_dir_mwt}/{video_id}')

    # Process video with MWT
    command = ['julia', '--project=.', f'src/mwt-cli.jl', video_path,
               f'{output_dir_mwt}/{video_id}', '--frame-rate', '30', '--pixel-thresholds', str(pixel_thr1),
               str(pixel_thr2), '--size-thresholds', str(size_thr1), str(size_thr2), '--pixel-size', '0.073',
               '--date-time', date_time]
    subprocess.run(command, cwd=working_dir, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Rename output file and move to output directory
    for file in os.listdir(f'{output_dir_mwt}/{video_id}/{date_time}'):
        shutil.move(f'{output_dir_mwt}/{video_id}/{date_time}/{file}',
                    f'{output_dir_mwt}/{video_id}/{file.replace("20", video_id)}')
    os.rmdir(f'{output_dir_mwt}/{video_id}/{date_time}')

Processing A1DF35_B1_2022-07-07-164359-0000_ffv1
Processing A1DF34_A1_2022-07-07-133023-0000_ffv1
Processing A1DF37_A1_2022-07-06-093303-0000_ffv1
Processing A1DF36_A1_2022-07-07-142945-0000_ffv1
Processing A1DF33_A1_2022-07-07-153358-0000_ffv1
Processing A1DF39_B1_2022-07-12-185350-0000_ffv1
Processing A1DF38_B1_2022-07-12-195403-0000_ffv1
Processing A1DF33_B1_2022-07-07-153521-0000_ffv1
Processing A1DF32_A1_2022-07-12-161124-0000_ffv1
Processing A1DF39_A1_2022-07-12-184950-0000_ffv1
Processing A1DF37_B1_2022-07-06-093415-0000_ffv1
Processing A1DF31_A1_2022-07-12-150920-0000_ffv1
Processing A1DF35_A1_2022-07-07-164027-0000_ffv1
Processing A1DF36_B1_2022-07-07-143248-0000_ffv1
Processing A1DF32_B1_2022-07-12-161457-0000_ffv1
Processing A1DF38_A1_2022-07-12-194950-0000_ffv1
Processing A1DF33_A2_2022-07-07-153918-0000_ffv1
Processing A1DF34_B1_2022-07-07-133354-0000_ffv1


# WF-NTP Tracker
We use the [WF-NTP command line version](https://github.com/Lilly-May/wf-ntp-cli) to process the videos. The tool must be installed first by cloning the linked repository. The installation instructions can be found in the repository. We expect a wf-ntp-cli directory as working directory, which is simply the cloned repository.

In [23]:
# Change the following paths to your needs

# Directory containing all videos to be processed
video_dir = '/Path/to/Larvae_Videos'
# Path to the mwt-cli installation; Installation instructions: https://github.com/Lilly-May/wf-ntp-cli
working_dir = '/Path/to/wf-ntp-cli'
# Directory to save the output spine files, the output for each video is saved in a subdirectory named after the video
output_dir_wf_ntp = '/Path/to/Larvae_Tracking_Benchmarking/Results/WF_NTP'

In [ ]:
# Optimized hyperparameters
threshold = 15
opening = 2
closing = 4
min_size = 31
max_size = 2558
minimum_ecc = 0.816162

hyperparams = f'--fps 30 --px_to_mm 0.073 --threshold {threshold} --opening {opening} --closing {closing} --min_size {min_size} --max_size {max_size} --minimum_ecc {minimum_ecc} --skeletonize True --do_full_prune True'.split(' ')

date_time = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')

# Iterate over all videos in video directory and process them
for file in os.listdir(video_dir):
    video_path = os.path.join(video_dir, file)
    video_id = os.path.basename(video_path).split('.')[0]
    print(f'Processing {video_id}')
    os.makedirs(f'{output_dir_wf_ntp}/{video_id}/{date_time}')

    # Process video with WF-NTP
    command = ['python', f'{working_dir}/src/wf_ntp_cli.py', video_path, f'{output_dir_wf_ntp}/{video_id}/{date_time}']
    command.extend(hyperparams)
    subprocess.run(command, cwd=working_dir, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Rename output file and move to output directory
    for file in os.listdir(f'{output_dir_wf_ntp}/{video_id}/{date_time}'):
        shutil.move(f'{output_dir_wf_ntp}/{video_id}/{date_time}/{file}', f'{output_dir_wf_ntp}/{video_id}/{file}')
    os.rmdir(f'{output_dir_wf_ntp}/{video_id}/{date_time}')

# Tierpsy Tracker
We use the [Tierpsy command line version](https://gitlab.com/larvataggerpipelines/tierpsy-cli) to process the videos. The tool must be installed first by cloning the linked repository. The installation instructions can be found in the repository. We expect a tierpsy directory as working directory, which is simply the cloned repository.

In [ ]:
# Change the following paths to your needs

# Directory containing all videos to be processed
video_dir = '/Path/to/Larvae_Videos'
# Path to the mwt-cli installation; Installation instructions: https://github.com/Lilly-May/wf-ntp-cli
working_dir = '/Path/to/tierpsy-cli'
# Directory to save the output spine files, the output for each video is saved in a subdirectory named after the video
output_dir_tierpsy = '/Path/to/Larvae_Tracking_Benchmarking/Results/Tierpsy'

In [ ]:
# Optimized hyperparameters
mask_min_area = 55
mask_max_area = 3014
thresh_C = 4
thresh_block_size = 60
dilation_size = 12
strel_size = 5
worm_bw_thresh_factor = 0.9648733750186274

date_time = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')

hyperparams = f'--frame-rate 30 --mask-min-area {mask_min_area} --mask-max-area {mask_max_area} --strel-size {strel_size} --worm-bw-thresh-factor {worm_bw_thresh_factor} --thresh-block-size {thresh_block_size} --dilation-size {dilation_size} --thresh-C {thresh_C} --pixel-size 0.073 --date-time {date_time}'.split(' ')

# Iterate over all videos in video directory and process them
for file in os.listdir(video_dir):
    video_path = os.path.join(video_dir, file)
    video_id = os.path.basename(video_path).split('.')[0]
    print(f'Processing {video_id}')
    os.mkdir(f'{output_dir_mwt}/{video_id}')

    command = ['julia', '--project=.', f'src/tierpsy-cli.jl', video_path,
                   f'{output_dir_tierpsy}/{video_id}/{date_time}']
    command.extend(hyperparams)
    subprocess.run(command, cwd=working_dir, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Rename output file and move to output directory
    for file in os.listdir(f'{output_dir_tierpsy}/{video_id}/{date_time}'):
        shutil.move(f'{output_dir_tierpsy}/{video_id}/{date_time}/{file}', f'{output_dir_tierpsy}/{video_id}/{file}')
    os.rmdir(f'{output_dir_tierpsy}/{video_id}/{date_time}')